In [9]:
from azure.identity import AzureCliCredential
from azure.mgmt.resource import ResourceManagementClient
from azure.mgmt.network import NetworkManagementClient
from azure.mgmt.compute import ComputeManagementClient
import os
from azure.identity import AzureCliCredential
from azure.mgmt.resource import SubscriptionClient

In [10]:
credential = AzureCliCredential()
subscription_client = SubscriptionClient(credential)

subscription_id = next(subscription_client.subscriptions.list())
subs_id = subscription_id.subscription_id
print(subs_id)

542414c9-86c5-4daf-9ef7-a109a8241211


In [11]:
# Step 1: Provision a resource group
resource_client = ResourceManagementClient(credential, subs_id)

In [12]:
RESOURCE_GROUP_NAME = "PythonAzureExample-VM-rg"
LOCATION = "westus2"

In [13]:
rg_result = resource_client.resource_groups.create_or_update(RESOURCE_GROUP_NAME,
    {
        "location": LOCATION
    }
)

In [14]:
print(f"Provisioned resource group {rg_result.name} in the {rg_result.location} region")

Provisioned resource group PythonAzureExample-VM-rg in the westus2 region


In [15]:
# Step 2: provision a virtual network
VNET_NAME = "python-example-vnet"
SUBNET_NAME = "python-example-subnet"
IP_NAME = "python-example-ip"
IP_CONFIG_NAME = "python-example-ip-config"
NIC_NAME = "python-example-nic"

In [16]:
network_client = NetworkManagementClient(credential, subs_id)

# Provision the virtual network and wait for completion
poller = network_client.virtual_networks.begin_create_or_update(RESOURCE_GROUP_NAME,
    VNET_NAME,
    {
        "location": LOCATION,
        "address_space": {
            "address_prefixes": ["10.0.0.0/16"]
        }
    }
)

In [17]:
vnet_result = poller.result()

In [18]:
print(f"Provisioned virtual network {vnet_result.name} with address prefixes {vnet_result.address_space.address_prefixes}") 
print(vnet_result)

Provisioned virtual network python-example-vnet with address prefixes ['10.0.0.0/16']
{'additional_properties': {}, 'id': '/subscriptions/542414c9-86c5-4daf-9ef7-a109a8241211/resourceGroups/PythonAzureExample-VM-rg/providers/Microsoft.Network/virtualNetworks/python-example-vnet', 'name': 'python-example-vnet', 'type': 'Microsoft.Network/virtualNetworks', 'location': 'westus2', 'tags': None, 'extended_location': None, 'etag': 'W/"ce15b4e5-0f61-4051-a916-fa2545f28edb"', 'address_space': <azure.mgmt.network.v2021_02_01.models._models_py3.AddressSpace object at 0x7f87d10ddfd0>, 'dhcp_options': None, 'flow_timeout_in_minutes': None, 'subnets': [], 'virtual_network_peerings': [], 'resource_guid': '730dcaff-04e5-42a4-b65e-b59c05bfa9e2', 'provisioning_state': 'Succeeded', 'enable_ddos_protection': False, 'enable_vm_protection': None, 'ddos_protection_plan': None, 'bgp_communities': None, 'ip_allocations': None}


In [19]:
from azure.mgmt.network.v2020_06_01.models import NetworkSecurityGroup
nsg_name = "testnsg"

nsg_params = NetworkSecurityGroup(id= "testnsg", location=LOCATION, tags={ "name" : "testnsg"})  

poller = network_client.network_security_groups.begin_create_or_update(RESOURCE_GROUP_NAME, "testnsg", parameters=nsg_params) 
nsg_result = poller.result()

print(f"{nsg_result}")

{'additional_properties': {}, 'id': '/subscriptions/542414c9-86c5-4daf-9ef7-a109a8241211/resourceGroups/PythonAzureExample-VM-rg/providers/Microsoft.Network/networkSecurityGroups/testnsg', 'name': 'testnsg', 'type': 'Microsoft.Network/networkSecurityGroups', 'location': 'westus2', 'tags': {'name': 'testnsg'}, 'etag': 'W/"c1e827a9-53c6-415b-ac93-dbf1aad15cd6"', 'security_rules': [], 'default_security_rules': [<azure.mgmt.network.v2021_02_01.models._models_py3.SecurityRule object at 0x7f87a0775550>, <azure.mgmt.network.v2021_02_01.models._models_py3.SecurityRule object at 0x7f87a0775850>, <azure.mgmt.network.v2021_02_01.models._models_py3.SecurityRule object at 0x7f87a07a1bb0>, <azure.mgmt.network.v2021_02_01.models._models_py3.SecurityRule object at 0x7f87a076af10>, <azure.mgmt.network.v2021_02_01.models._models_py3.SecurityRule object at 0x7f87a076a910>, <azure.mgmt.network.v2021_02_01.models._models_py3.SecurityRule object at 0x7f87a076afd0>], 'network_interfaces': None, 'subnets': No

In [20]:
from azure.mgmt.network.v2020_06_01.models import SecurityRule
sec_rule_name = "testrule"
sec_rule_params = SecurityRule(
    id="testrule", 
    name=sec_rule_name, 
    protocol="Tcp", 
    source_port_range="*",
    destination_port_range="22",
    source_address_prefix="*",
    destination_address_prefix="*",
    access="Allow",
    direction="inbound", 
    priority=100,
)

poller = network_client.security_rules.begin_create_or_update(RESOURCE_GROUP_NAME,nsg_name,security_rule_name="testrule",security_rule_parameters=sec_rule_params) 
sec_rule_result = poller.result()
print(f"{sec_rule_result}")

{'additional_properties': {}, 'id': '/subscriptions/542414c9-86c5-4daf-9ef7-a109a8241211/resourceGroups/PythonAzureExample-VM-rg/providers/Microsoft.Network/networkSecurityGroups/testnsg/securityRules/testrule', 'name': 'testrule', 'etag': 'W/"0384df35-4bc4-4f14-8d13-3e99354fd7c1"', 'type': 'Microsoft.Network/networkSecurityGroups/securityRules', 'description': None, 'protocol': 'Tcp', 'source_port_range': '*', 'destination_port_range': '22', 'source_address_prefix': '*', 'source_address_prefixes': [], 'source_application_security_groups': None, 'destination_address_prefix': '*', 'destination_address_prefixes': [], 'destination_application_security_groups': None, 'source_port_ranges': [], 'destination_port_ranges': [], 'access': 'Allow', 'priority': 100, 'direction': 'Inbound', 'provisioning_state': 'Succeeded'}


In [21]:
# step 3: Provision the subnet and wait for completion
poller = network_client.subnets.begin_create_or_update(RESOURCE_GROUP_NAME, 
    VNET_NAME, SUBNET_NAME,                                                 
    { "address_prefix": "10.0.0.0/24", 
     "network_security_group":{
        "id": nsg_result.id,
    } },
)
subnet_result = poller.result()

print(f"Provisioned virtual subnet {subnet_result.name} with address prefix {subnet_result.address_prefix}")


Provisioned virtual subnet python-example-subnet with address prefix 10.0.0.0/24


In [22]:
print(subnet_result)

{'additional_properties': {}, 'id': '/subscriptions/542414c9-86c5-4daf-9ef7-a109a8241211/resourceGroups/PythonAzureExample-VM-rg/providers/Microsoft.Network/virtualNetworks/python-example-vnet/subnets/python-example-subnet', 'name': 'python-example-subnet', 'etag': 'W/"0314701b-8b94-47e7-ba05-85b87b986059"', 'type': 'Microsoft.Network/virtualNetworks/subnets', 'address_prefix': '10.0.0.0/24', 'address_prefixes': None, 'network_security_group': <azure.mgmt.network.v2021_02_01.models._models_py3.NetworkSecurityGroup object at 0x7f87a095a7c0>, 'route_table': None, 'nat_gateway': None, 'service_endpoints': None, 'service_endpoint_policies': None, 'private_endpoints': None, 'ip_configurations': None, 'ip_configuration_profiles': None, 'ip_allocations': None, 'resource_navigation_links': None, 'service_association_links': None, 'delegations': [], 'purpose': None, 'provisioning_state': 'Succeeded', 'private_endpoint_network_policies': 'Enabled', 'private_link_service_network_policies': 'Enabl

In [23]:
# Step 4: Provision an IP address and wait for completion
poller = network_client.public_ip_addresses.begin_create_or_update(RESOURCE_GROUP_NAME,
    IP_NAME,
    {
        "location": LOCATION,
        "sku": { "name": "Standard" },
        "public_ip_allocation_method": "Static",
        "public_ip_address_version" : "IPV4"
    }
)

In [24]:
ip_address_result = poller.result()
print(f"Provisioned public IP address {ip_address_result.name} with address {ip_address_result.ip_address}")

Provisioned public IP address python-example-ip with address 52.183.13.166


In [25]:
# Step 5: Provision the network interface client
poller = network_client.network_interfaces.begin_create_or_update(RESOURCE_GROUP_NAME,
    NIC_NAME, 
    {
        "location": LOCATION,
        "ip_configurations": [ {
            "name": IP_CONFIG_NAME,
            "subnet": { "id": subnet_result.id },
            "public_ip_address": {"id": ip_address_result.id }
        }]
    }
)

In [26]:
nic_result = poller.result()
print(f"Provisioned network interface client {nic_result.name}")

Provisioned network interface client python-example-nic


In [27]:
# Step 6: Provision the virtual machine
compute_client = ComputeManagementClient(credential, subs_id)

VM_NAME = "ExampleVM"
USERNAME = "azureuser"
PASSWORD = "ChangePa$$w0rd24"

In [28]:
poller = compute_client.virtual_machines.begin_create_or_update(RESOURCE_GROUP_NAME, VM_NAME,
                                                               {
                                                                   "location": LOCATION,
                                                                   "storage_profile":{
                                                                       "image_reference":{
                                                                           "publisher":'Canonical',
                                                                           "offer":"UbuntuServer",
                                                                           "sku":"16.04.0-LTS",
                                                                           "version":"latest"
                                                                       }
                                                                   },
                                                                   "hardware_profile":{
                                                                       "vm_size":"Standard_DS1_v2"
                                                                   },
                                                                   "os_profile":{
                                                                       "computer_name":VM_NAME,
                                                                       "admin_username":USERNAME,
                                                                       "admin_password":PASSWORD
                                                                   },
                                                                   "network_profile":{
                                                                       "network_interfaces":[{
                                                                           "id":nic_result.id,
                                                                       }]
                                                                   },
                                                               })

In [29]:
vm_result = poller.result()

In [30]:
print(f"Provisioned virtual machine{vm_result.name}")

Provisioned virtual machineExampleVM
